In [7]:
import pandas as pd
from numpy import nan

In [3]:
# importing with pandas assuming accurate headers and reasonable size of data
raw_data = pd.read_csv("test-data.csv")

In [4]:
# some failure testing, do not include in live
raw_data = raw_data.append(pd.read_csv("fail_tests.csv"))

In [5]:
raw_data

,Account ID,Account Name,First Name,Created On
0,314159,superman,Ka-el,1/12/13
1,271,batman,Bruce,11/19/16
2,8675309,hulk,Bruce,2/22/99
3,99999,batgirl,Yvonne,3/5/13
0,asdf,thisEntryShouldBeDeleted,NaN,None
1,NaN,thisEntryShouldBeDeleted,NaN,whenever
2,NaN,thisEntryShouldBeDeleted,NaN,2/4/12
3,271,thisEntryShouldBeDeleted,NaN,11/19/16
4,21,NaN,NaN,NaN


In [8]:
clean_data = raw_data

# might not necessarily want to assume the type of the Account ID, 
# but definitely dont want .0s on the end for api query
# coercing yields NaNs where errors occur which are filtered later
clean_data["Account ID"] = pd.to_numeric(clean_data["Account ID"], errors='coerce')\
    .astype("int", errors='ignore')#, downcast="integer")
# set as datetime to check validity and match "Status Set On" data
clean_data["Created On"] = pd.to_datetime(clean_data["Created On"], errors='coerce')

#unneccesary column
clean_data = clean_data.drop(["Account Name"], axis=1)

# clean data, create contract account_id is unique integer not na, and one other property must be non na.
clean_data = clean_data.dropna(axis = 0, how="any",subset=["Account ID"]) # must have Account ID
clean_data = clean_data.dropna(axis = 0, how="all",subset=["First Name","Created On"]) # account with all empty data
clean_data["Account ID"] = clean_data["Account ID"].astype("int", errors='ignore')

# create columns for output
clean_data["Status"] = nan
clean_data["Status Set On"] = nan

# account id acts as index and must be unique
clean_data = clean_data.drop_duplicates(subset=["Account ID"])
clean_data = clean_data.set_index("Account ID")
clean_data

,First Name,Created On,Status,Status Set On
Account ID,,,,
314159,Ka-el,2013-01-12,NaN,NaN
271,Bruce,2016-11-19,NaN,NaN
8675309,Bruce,1999-02-22,NaN,NaN
99999,Yvonne,2013-03-05,NaN,NaN
